# 2022年度第4ターム「実験数学D」 第04回 01/10(火)4限

In [ ]:
# 必要なモジュールをインストールする
%pip install pulp numpy

In [2]:
# 必要なモジュールをインポートする
import numpy as np
import scipy
from nptyping import Float, NDArray, Shape

以下の制約条件なしの非線型最適化問題 (最小化) を解く．ここで $x = {}^{T}(x_{1}, x_{2})$ である

## 1.

- 目的関数: $f(\bm{x}) = x_{1}^{2} + x_{2}^{2} - x_{3}^{2} + 4x_{2}x_{3} - 3x_{1} + 2x_{2} + x_{3} - 6$
- 最適解は $(x_{1}, x_{2}, x_{3}) = (\frac{7}{6}, -\frac{4}{3}, \frac{1}{6})$ で，最適値は $-9$ ．

## 2.

- 目的関数: $f(\bm{x}) = 100(x_{1}^{2} + x_{2})^{2} + (x_{1} - 1)^{2} + 100(x_{2}^{2} - x_{3}^{2})^{2} + (x_{2} - 1)^{2}$
- 最適解は $(x_{1}, x_{2}, x_{3}) = (1, 1, 1)$ で，最適値は $0$ ．

Newton法を用いる．

In [3]:
# 2. 次の制約条件なしの非線型最適化問題 (最小化) をNewton法で解く. ここでx = T(x1, x2, x3)である:
# 目的関数: f(x) = 100(x1^2 + x2)^2 + (x1 - 1)^2 + 100(x2^2 - x3^2)^2 + (x2 - 1)^2
# 最適解: (x1, x2, x3) = (1, 1, 1)
# 最適値: 0
def f(xs: NDArray[Shape["3"], Float]) -> float:
    """
    目的関数
    """
    return sum(100 * (xs[i + 1] - xs[i] ** 2) ** 2 + (xs[i] - 1) ** 2 for i in range(2))


def grad_f(xs: NDArray[Shape["3"], Float]) -> NDArray[Shape["3"], Float]:
    """
    目的関数の勾配ベクトル
    """
    return np.array(
        [
            -400 * xs[0] * (-xs[0] ** 2 + xs[1]) + 2 * xs[0] - 2,
            -200 * xs[0] ** 2 - 400 * xs[1] * (-xs[1] ** 2 + xs[2]) + 202 * xs[1] - 2,
            -200 * xs[1] ** 2 + 200 * xs[2],
        ]
    )


def hessian_f(xs: NDArray[Shape["3"], Float]) -> NDArray[Shape["3, 3"], Float]:
    """
    目的関数のヘッセ行列
    """
    return np.array(
        [
            [1200 * xs[0] ** 2 - 400 * xs[1] + 2, -400 * xs[0], 0],
            [-400 * xs[0], 1200 * xs[1] ** 2 - 400 * xs[2] + 202, -400 * xs[1]],
            [0, -400 * xs[1], 200],
        ]
    )


# 初期点を設定する. ただし, 各成分の型にnp.float64を指定する
xs0 = np.array([10, 10, 10], dtype=np.float64)
# 許容誤差
eps = 1e-6
# 最大反復回数
iter_max = 100
# Newton法で最適解を求める
while scipy.linalg.norm(grad_f(xs0)) > eps:
    d = -np.dot(scipy.linalg.inv(hessian_f(xs0)), grad_f(xs0))
    xs0 += d
# 結果を表示する
print(f"最適解: x = {xs0}")
print(f"最適値: {f(xs0)}")

最適解: x = [1. 1. 1.]
最適値: 1.2766836147122762e-19


## 3.

- 目的関数: $f(\bm{x}) = (1.5 - x_{1} + x_{1}x_{2})^{2} + (2.25 - x_{1} + x_{1}x_{2}^{2})^{2} + (2.625 - x_{1} + x_{1}x_{2}^{3})^{2}$ の最小化
- 最適解は $(x_{1}, x_{2}) = (3, 0.5)$ で，最適値は $0$ ．

準Newton法を用いる．

In [4]:
# 3. 次の制約条件なしの非線型最適化問題 (最小化) を準Newton法で解く. ここでx = T(x1, x2)である:
# 目的関数: f(x) = (1.5 - x1 + x1x2)^2 + (2.25 - x1 + x1x2^2)^2 + (2.625 - x1 + x1x2^3)^2
# 最適解: (x1, x2) = (3, 0.5)
# 最適値: 0
def f(xs: NDArray[Shape["3"], Float]) -> float:
    """
    目的関数
    """
    return (
        (1.5 - xs[0] + xs[0] * xs[1]) ** 2
        + (2.25 - xs[0] + xs[0] * (xs[1] ** 2)) ** 2
        + (2.625 - xs[0] + xs[0] * (xs[1] ** 3)) ** 2
    )


# 初期点を設定する
xs0 = np.array([0, 0])
# 準Newton法で大域的最適解を求める
ans = scipy.optimize.minimize(f, xs0, method="BFGS")
# 結果を表示する
print(ans)
print(f"最適解: x = {ans.x}")
print(f"最適値: {ans.fun}")

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 9.027328141904564e-15
        x: [ 3.000e+00  5.000e-01]
      nit: 13
      jac: [-1.339e-07  1.133e-06]
 hess_inv: [[ 3.235e+00  8.083e-01]
            [ 8.083e-01  2.239e-01]]
     nfev: 48
     njev: 16
最適解: x = [3.00000012 0.50000005]
最適値: 9.027328141904564e-15
